In [1]:
import pandas as pd
# import unicodecsv as csv
import re
import nltk
# from haversine import haversine, Unit
from collections import Counter,defaultdict
import re,string
import numpy as np
from nltk.corpus import stopwords
# words = stopwords.words('english')
from datetime import datetime
import re
import ast

import pandas as pd
import time
import sys
import spacy
from datetime import datetime
import os
from tqdm import tqdm

In [3]:
# data0=pd.read_csv("golden_test_v1_detect_revised.tsv",sep='\t')
data0=pd.read_excel("golden_test_v1_detect_revised.xlsx")
data0

,PMID,total_sentences,sentence_order,sentence,detected_candidate,tagged_sentences,category,revised_candidate
0,24390784,5.0,1.0,"Research in Tiriki, Western Province, Kenya, i...",Tiriki,"Research in [locB] Tiriki [locE], Western Prov...",1,Tiriki
1,24390784,5.0,1.0,"Research in Tiriki, Western Province, Kenya, i...",Western Province,"Research in Tiriki, [locB] Western Province [l...",1,NaN
2,24390784,5.0,1.0,"Research in Tiriki, Western Province, Kenya, i...",Kenya,"Research in Tiriki, Western Province, [locB] K...",1,Kenya
3,24390784,6.0,4.0,ka]Key Words kb]age grading kb]senior age grou...,Kenya,ka]Key Words kb]age grading kb]senior age grou...,1,Kenya
4,24390784,6.0,4.0,ka]Key Words kb]age grading kb]senior age grou...,Africa,ka]Key Words kb]age grading kb]senior age grou...,1,Africa
...,...,...,...,...,...,...,...,...
601,27449091,12.0,4.0,"In stage IV disease, the sex disparity favorin...",IV,"In stage [locB] IV [locE] disease, the sex dis...",0,IV
602,27449091,12.0,6.0,Hispanics and Native Americans with locoregion...,Whites,Hispanics and Native Americans with locoregion...,0,Whites
603,27449091,12.0,7.0,"Additionally, Asians with stage I-III RC had s...",Whites,"Additionally, Asians with stage I-III RC had s...",0,Whites
604,27451861,10.0,5.0,The ICU (n = 49) and no-ICU cohorts (n = 47) w...,NS,The ICU (n = 49) and no-ICU cohorts (n = 47) w...,0,NS


In [3]:
data0 = data0.drop('revised_candidate', axis=1)
data0 = data0.rename(columns={'detected_candidate': 'candidate'})

In [8]:
data0 = data0.rename(columns={'sentence': 'sentences'})

In [4]:
cand_length = []
cand_cap_num = []
cand_cap_ratio = []
cand_has_num = []
cand_has_punc = []
cand_capitalized = []

def contains_punctuation(s):
    return any(not c.isalnum() and not c.isspace() for c in s)

for idx,sentence in enumerate(data0['tagged_sentences']):
    loc_start = sentence.find('[locB]') + 6  # skip [locB]
    loc_end = sentence.find('[locE]')
    location = sentence[loc_start:loc_end].strip()  # remove space on both sides
    
    # lengh of candidate
    length = len(location)
    cand_length.append(length)
    
    # capitalized letters
    cap_num = sum(1 for c in location if c.isupper())
    cand_cap_num.append(cap_num)
    
    # ratio of capitalized letters
    cap_ratio = cap_num / length if length > 0 else 0
    cand_cap_ratio.append(cap_ratio)
    
    # contains number?
    has_num = 1 if any(c.isdigit() for c in location) else 0
    cand_has_num.append(has_num)
    
    # contains punc
    has_punc = 1 if contains_punctuation(location) else 0
    cand_has_punc.append(has_punc)
    
    # 1st letter capitalized 
    try:
        capitalized = 1 if location[0].isupper() else 0
        cand_capitalized.append(capitalized)
    except:
        print(idx,location)
        cand_capitalized.append(0)

# show results
(cand_length[:3], cand_cap_num[:3], cand_cap_ratio[:3], cand_has_num[:3], cand_has_punc[:3], cand_capitalized[:3])


([6, 16, 5],
 [1, 2, 1],
 [0.16666666666666666, 0.125, 0.2],
 [0, 0, 0],
 [0, 0, 0],
 [1, 1, 1])

In [5]:
len(cand_length),len(cand_cap_num),len(cand_cap_ratio),len(cand_has_num),len(cand_has_punc),len(cand_capitalized)

(606, 606, 606, 606, 606, 606)

In [6]:
data0["cand_length"]=cand_length
data0["cand_cap_num"]=cand_cap_num
data0["cand_cap_ratio"]=cand_cap_ratio
data0["cand_has_num"]=cand_has_num
data0["cand_has_punc"]=cand_has_punc
data0["cand_capitalized"]=cand_capitalized

In [9]:
cand_words = []
cand_etal = []
comma_before = []
comma_after = []
parentheses_before = []
parentheses_after = []
year_before = []
year_after = []
etal_before = []
etal_after = []

   
for sentence in data0['sentences']:
    cand_words.append(len(location.split()))
    cand_etal.append(1 if 'et al' in location.lower() else 0)

def find_symbol_distance(sentence, symbol, loc_index, is_before=True):
    """calculate distanc of punc before or after [locB]/[locE]"""
    if is_before:
        segment = sentence[:loc_index].split()
        segment.reverse()  
    else:
        segment = sentence[loc_index:].split()
    
    for i, word in enumerate(segment, start=1):
        if symbol in word:
            return i
    return 0

def find_brackets_distance(sentence, loc_index, is_before=True):
    """calculate distanc of parentheses before or after [locB]/[locE]/"""
    brackets = ['(', ')']
    distance = float('inf')
    for bracket in brackets:
        d = find_symbol_distance(sentence, bracket, loc_index, is_before)
        distance = min(d, distance)
    return distance if distance != float('inf') else 0

def find_years_distance_corrected(sentence, loc_index, is_before=True):
    """calculate distanc of year before or after [locB]/[locE]"""
    pattern = r'\b(1|2)\d{3}\b'
    if is_before:
        segment = sentence[:loc_index].split()
        segment.reverse()  # reverse
    else:
        segment = sentence[loc_index:].split()
    
    for i, word in enumerate(segment, start=1):
        if re.search(pattern, word):
            return i
    return 0

def find_etal_distance_corrected(sentence, loc_index, is_before=True):
    """calculate distanc of et al before or after [locB]/[locE]"""
    etal = 'et al'
    if is_before:
        segment = ' '.join(sentence[:loc_index].lower().split())  
        segment = segment.replace('et al', 'et_al')  
        segment = segment.split()
        segment.reverse()  
    else:
        segment = ' '.join(sentence[loc_index:].lower().split()) 
        segment = segment.replace('et al', 'et_al')  
        segment = segment.split()
    
    for i, phrase in enumerate(segment, start=1):
        if 'et_al' in phrase:
            return i
    return 0

for sentence in data0['tagged_sentences']:
    loc_start = sentence.find('[locB]')
    loc_end = sentence.find('[locE]') + len('[locE]')
    
    comma_before.append(find_symbol_distance(sentence, ',', loc_start, is_before=True))
    comma_after.append(find_symbol_distance(sentence, ',', loc_end, is_before=False))
    parentheses_before.append(find_brackets_distance(sentence, loc_start, is_before=True))
    parentheses_after.append(find_brackets_distance(sentence, loc_end, is_before=False))
    year_before.append(find_years_distance_corrected(sentence, loc_start, is_before=True))
    year_after.append(find_years_distance_corrected(sentence, loc_end, is_before=False))
    etal_before.append(find_etal_distance_corrected(sentence, loc_start, is_before=True))
    etal_after.append(find_etal_distance_corrected(sentence, loc_end, is_before=False))

    
(cand_words[:3], cand_etal[:3], comma_before[:3], comma_after[:3], parentheses_before[:3], parentheses_after[:3], year_before[:3], year_after[:3], etal_before[:3], etal_after[:3])


([1, 1, 1],
 [0, 0, 0],
 [0, 1, 1],
 [1, 1, 1],
 [0, 0, 0],
 [0, 0, 0],
 [0, 0, 0],
 [6, 4, 3],
 [0, 0, 0],
 [0, 0, 0])

In [10]:
data0["cand_words"]=cand_words
data0["cand_etal"]=cand_etal
data0["comma_before"]=comma_before
data0["comma_after"]=comma_after
data0["parentheses_before"]=parentheses_before
data0["parentheses_after"]=parentheses_after
data0["year_before"]=year_before
data0["year_after"]=year_after
data0["etal_before"]=etal_before
data0["etal_after"]=etal_after

In [11]:
cand_pos = []

for index, row in data0.iterrows():
    sentence = row['tagged_sentences']
    candidate_tagged = f"[locB] {row['candidate']} [locE]"
    parts = sentence.split(candidate_tagged)
    words_before = len(re.sub(r'\[locB\]|\[locE\]', '', parts[0]).split())
    candidate_position = words_before + 1  
    total_words = len(re.sub(r'\[locB\]|\[locE\]', '', sentence).split()) + 1 
    relative_pos = candidate_position / total_words
    cand_pos.append(relative_pos)


In [12]:
data0["cand_pos"]=cand_pos

In [23]:
nonUScities = {}

with open('mapaffil_name_variants_nonUScities.tsv', 'r',encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split('\t')
        coord, full_location = parts[:2]
        location_parts = full_location.split(', ')
        if location_parts[0]=="Russia":
            country = location_parts[0]
            province = location_parts[1] if len(location_parts) > 1 else ''
            city = ', '.join(location_parts[2:]) if len(location_parts) > 2 else ''
        else:
            country = location_parts[-1]
            province = location_parts[-2] if len(location_parts) > 1 else ''
            city = ', '.join(location_parts[:-2]) if len(location_parts) > 2 else ''

        for variant in parts[2:]:
            clean_variant = re.sub(r'\s*\d+\s*', '', variant).strip()
            nonUScities[clean_variant] = [coord, city, province, country]


In [25]:
US_states = {}

with open('mapaffil_name_variants_US_states.tsv', 'r',encoding='utf-8') as file:
    for line in file:
        state_list = line.strip().split('\t')
        
        if state_list:
            US_states[state_list[0]] = state_list

reversed_states = {}


for key, values in US_states.items():
    for value in values:
        reversed_states[value] = key


In [26]:
for key in reversed_states:
    state = reversed_states[key]
    reversed_states[key] = ['', '', state, 'USA']


In [27]:
countries = {}

with open('mapaffil_name_variants_countries.tsv', 'r',encoding='utf-8') as file:
    for line in file:
        countries_list = line.strip().split('\t')
        
        if countries_list:
            countries[countries_list[0]] = countries_list

countries["USA"].append("the United States")
countries["UK"].append("the United Kingdom")
countries["USA"].append("U.S.")


reversed_countries = {}

for key, values in countries.items():
    for value in values:
        reversed_countries[value] = key


In [29]:
for key in reversed_countries:
    country = reversed_countries[key]
    reversed_countries[key] = ['', '', '', country]


In [30]:
UScities = {}

with open('mapaffil_name_variants_UScities.tsv', 'r',encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split('\t')
        coord, state, city_name = parts[:3]

        for variant in parts[3:]:
            clean_variant = re.sub(r'\s*\d+\s*', '', variant).strip()
            UScities[clean_variant] = [coord, state, city_name]


In [31]:
for city in UScities:
    coord = UScities[city][0]
    state = UScities[city][1]
    city_name = UScities[city][2]
    UScities[city] = [coord, city_name, state, 'USA']

In [32]:
res_list = []
mapaffil_matched = []

for index, row in data0.iterrows():
    temp = []
    candidate = row['candidate']
    
    for d in [UScities, nonUScities, reversed_countries, reversed_states]:
        if candidate in d:
            temp.append(d[candidate])
    
    res_list.append(temp)
    
    mapaffil_matched.append(1 if temp else 0)

In [35]:
data0["og_mapaffil_res"]=res_list
data0["MapAffil_dict_matched"]=mapaffil_matched

In [36]:
def read_geolocations(filename):
    res=[]
    with open(filename) as file:
        for line in file:
            if len(line.strip())>0:
                res.append(line.strip())
    return res

Africa_Central=['Cameroon','Central African Republic','Chad','Congo','Democratic Republic of the Congo','Equatorial Guinea','Gabon','Sao Tome and Principe']
Africa_Eastern=['Burundi','Djibouti','Eritrea','Ethiopia','Kenya','Rwanda','Somalia','South Sudan','Sudan','Tanzania','Uganda']
Africa_Southern=['Angola','Botswana','Eswatini','Lesotho','Malawi','Mozambique','Namibia','South Africa','Zambia','Zimbabwe']
Africa_Western=['Benin','Benin','Burkina Faso','Cabo Verde',"Cote d'Ivoire","Gambia","Gambia","Ghana","Guinea","Guinea-Bissau","Liberia","Mali","Mauritania","Niger","Nigeria","Senegal","Sierra Leone","Togo"]
Africa_Northern=["Algeria","Egypt","Libya","Morocco","Tunisia"]

Africa = Africa_Central+Africa_Eastern+Africa_Southern+Africa_Western+Africa_Northern

Caribbean_Region=read_geolocations("Caribbean_Region.txt")
Central_America=read_geolocations("Central_America.txt")
South_America=read_geolocations("South_America.txt")
North_America=read_geolocations("North_America.txt")
United_States=read_geolocations("United_States.txt")

Americas=Caribbean_Region+Central_America+South_America+North_America+United_States

Asia=read_geolocations("Asia.txt")
Others=read_geolocations("Others.txt")
Locations=Africa+Americas+Asia+Others

Canada_states={
    "AB":"Alberta",
    "BC":"British Columbia",
    "MB":"Manitoba",
    "NB":"New Brunswick",
    "NL":"Newfoundland and Labrador",
    "NT":"Northwest Territories",
    "NS":"Nova Scotia",
    "NU":"Nunavut",
    "ON":"Ontario",
    "PE":"Prince Edward Island",
    "QC":"Quebec",
    "SK":"Saskatchewan",
    "YT":"Yukon Territory"
}

USA_states={"AL":"Alabama","AK":"Alaska","AZ":"Arizona","AR":"Arkansas","CA":"California","CO":"Colorado","CT":"Connecticut","DE":"Delaware","FL":"Florida","GA":"Georgia","HI":"Hawaii","ID":"Idaho","IL":"Illinois","IN":"Indiana","IA":"Iowa","KS":"Kansas","KY":"Kentucky","LA":"Louisiana","ME":"Maine","MD":"Maryland","MA":"Massachusetts","MI":"Michigan","MN":"Minnesota","MS":"Mississippi","MO":"Missouri","MT":"Montana","NE":"Nebraska","NV":"Nevada","NH":"New Hampshire","NJ":"New Jersey","NM":"New Mexico","NY":"New York","NC":"North Carolina","ND":"North Dakota","OH":"Ohio","OK":"Oklahoma","OR":"Oregon","PA":"Pennsylvania","RI":"Rhode Island","SC":"South Carolina","SD":"South Dakota","TN":"Tennessee","TX":"Texas","UT":"Utah","VT":"Vermont","VA":"Virginia","WA":"Washington","WV":"West Virginia","WI":"Wisconsin","WY":"Wyoming"}

Australia_states={"ACT":"Australian Capital Territory",
                  "NSW":"New South Wales",
                  "NT":"Northern Territory",
                  "QLD":"Queensland",
                  "SA":"South Australia",
                  "TAS":"Tasmania",
                  "VIC":"Victoria",
                  "WA":"Western Australia",
                  "Norfolk Island":"Norfolk Island"
                 }

USA_states["DC"]="District of Columbia"
USA_states["PR"]="Puerto Rico"
USA_states["UM"]="U.S. Minor Outlying Islands"
USA_states["MP"]="Northern Mariana Islands"
USA_states["AS"]="American Samoa"
USA_states["VI"]="Virgin Islands"
USA_states["GU"]="Guam"
USA_states["AA"]="Armed Forces America"
USA_states["AE"]="Armed Forces"

In [37]:
mapcountry_list,mapstate_list=[],[]

# for i in data0["mapaffil_res"]:# res_list
for idx,i in enumerate(res_list):# res_list    
    temp_country,temp_state=[],[]
    if type(i)!=list:
        temp_list=ast.literal_eval(i)
    else:
        temp_list=i
    for res in temp_list:
        if res[-1]=="UK":
            temp_country.append('United Kingdom')
        elif res[-1]=="USA":
            temp_country.append('United States')
        else:
            temp_country.append(res[-1])
        if len(res[-2])==0:
            pass
        else:
            if res[-1]=="USA":
                if res[-2] in USA_states.keys():
                    temp_state.append(USA_states[res[-2]])
                else:
                    print(idx,"USA,",res[-2])
            elif res[-1]=="Canada":
                if res[-2] in Canada_states.keys():
                    temp_state.append(Canada_states[res[-2]])
                else:
                    print("Canada,",res[-2])
            elif res[-1]=="Australia":
                if res[-2] in Australia_states.keys():
                    temp_state.append(Australia_states[res[-2]])
                else:
                    print("Australia,",res[-2])
            else:
                temp_state.append(res[-2])
    mapcountry_list.append(temp_country)   
    mapstate_list.append(temp_state)

In [40]:
data0["mapstand_state_list"]=mapstate_list

In [41]:
standard_country_dict={
    "Palestine":"Middle East",
    "Viet Nam":"Vietnam",
    "Democratic Republic of Congo":"Congo",
    "North Korea":"Democratic People's Republic of Korea",
    "Sultanate of Oman":"Oman",
    "Republic of Georgia":"Georgia (Republic)",
    "East Timor":"Timor-Leste",
    "Niue":"Polynesia",
    "Nauru":"Micronesia",
    "Solomon Islands":"Melanesia",
    "Macao":"China",
    "Moskva":"Russia",
    "Belarus":"Republic of Belarus",
    "Mayotte":"Comoros",
    "Cook Islands":"Polynesia",
    "Macedonia":"Europe, Eastern",
    "French Polynesia":"Polynesia",
    "Kiribati":"Micronesia",
    "Tuvalu":"Micronesia",
    "Marshall Islands":"Micronesia",
    "Maldives":"Asia", 
    "Saint Martin":"West Indies",
    "Aland Islands":"Finland",
    "Cape Verde":"Cabo Verde",
    "Holy See":"Vatican City",
    "Faeroe Islands":"Denmark",
    "Isle of Man":"United Kingdom",
    "Wallis and Futuna":"Polynesia"
}


In [42]:
with open("mtrees2018.bin", "r", encoding="utf-8") as file:
    content = file.read()

In [43]:
lines = content.strip().split("\n")

names2code_dict = {}
code2names_dict = {}
hierarchy_dict = {}

for line in lines:
    name, code = line.split(";")
    names2code_dict[name.strip()] = code
    code2names_dict[code] = name.strip()
    parts = code.split(".")
    for i in range(1, len(parts)):
        key = ".".join(parts[:i+1])
        value = [".".join(parts[:j+1]) for j in range(i)]
        hierarchy_dict[key] = value

In [44]:
for countries in mapcountry_list:
    for country in countries:
        if country not in names2code_dict.keys():
            if country not in standard_country_dict.keys():
                print(country)

In [45]:
region_list,mapstand_country_list=[],[]
for country_list in mapcountry_list:
    temp,temp_country=[],[]
    for country in country_list:
        if country not in names2code_dict.keys():
            if country in standard_country_dict.keys():
                temp_country.append(country)
                for i in hierarchy_dict[names2code_dict[standard_country_dict[country]]]:
                    temp.append(code2names_dict[i])
            else:
                print(country)
        else:
            temp_country.append(country)
            for i in hierarchy_dict[names2code_dict[country]]:
                temp.append(code2names_dict[i])
    region_list.append(temp)
    mapstand_country_list.append(temp_country)

In [49]:
data0["rigion_list"]=region_list
data0["mapstand_country_list"]=mapstand_country_list

In [50]:
import json
with open('lastname_dict.json', 'r') as file:
    lastname_dict = json.load(file)

In [51]:
lastname_freq=[]
for cand in data0["candidate"]:
    if cand in lastname_dict:
        lastname_freq.append(lastname_dict[cand]/52966856)
    else:
        lastname_freq.append(0)

In [52]:
data0["lastname_freq"]=lastname_freq

In [53]:
with open('firstname_dict.json', 'r') as file:
    firstname_dict = json.load(file)

In [54]:
firstname_freq=[]
for cand in data0["candidate"]:
    if cand in firstname_dict:
        firstname_freq.append(firstname_dict[cand]/52966856)
    else:
        firstname_freq.append(0)

In [55]:
data0["firstname_freq"]=firstname_freq

In [56]:
with open('affiliation_dict.json', 'r') as file:
    affiliation_dict = json.load(file)

In [57]:
affiliation_freq=[]
for cand in data0["candidate"]:
    if cand in affiliation_dict:
        affiliation_freq.append(affiliation_dict[cand]/52966856)
    else:
        affiliation_freq.append(0)

In [58]:
data0["affiliation_freq"]=affiliation_freq

In [59]:
with open('title_dict.json', 'r') as file:
    title_dict = json.load(file)

In [60]:
title_freq=[]
for cand in data0["candidate"]:
    if cand in title_dict:
        title_freq.append(title_dict[cand]/15543305)
    else:
        title_freq.append(0)

In [61]:
data0["title_freq"]=title_freq

In [62]:
with open('abstract_dict.json', 'r') as file:
    abstract_dict = json.load(file)

In [63]:
abstract_freq=[]
for cand in data0["candidate"]:
    if cand in abstract_dict:
        abstract_freq.append(abstract_dict[cand]/15543305)
    else:
        abstract_freq.append(0)

In [64]:
data0["abstract_freq"]=abstract_freq

In [76]:
data0['total_sentences'].fillna(value=0, inplace=True)
data0['sentence_order'].fillna(value=0, inplace=True)

In [78]:
sum(data0.isna().any())

0

In [80]:
data0.columns

Index(['PMID', 'total_sentences', 'sentence_order', 'sentences', 'candidate',
       'tagged_sentences', 'category', 'cand_length', 'cand_cap_num',
       'cand_cap_ratio', 'cand_has_num', 'cand_has_punc', 'cand_capitalized',
       'cand_words', 'cand_etal', 'comma_before', 'comma_after',
       'parentheses_before', 'parentheses_after', 'year_before', 'year_after',
       'etal_before', 'etal_after', 'cand_pos', 'og_mapaffil_res',
       'MapAffil_dict_matched', 'mapstand_state_list', 'rigion_list',
       'mapstand_country_list', 'lastname_freq', 'firstname_freq',
       'affiliation_freq', 'title_freq', 'abstract_freq'],
      dtype='object')

In [81]:
data0['lastname_freq_log10'] = np.where(data0['lastname_freq'] <= 0, data0['lastname_freq'], np.log10(data0['lastname_freq']))
data0['firstname_freq_log10'] = np.where(data0['firstname_freq'] <= 0, data0['firstname_freq'], np.log10(data0['firstname_freq']))
data0['affiliation_freq_log10'] = np.where(data0['affiliation_freq'] <= 0, data0['affiliation_freq'], np.log10(data0['affiliation_freq']))
data0['title_freq_log10'] = np.where(data0['title_freq'] <= 0, data0['title_freq'], np.log10(data0['title_freq']))
data0['abstract_freq_log10'] = np.where(data0['abstract_freq'] <= 0, data0['abstract_freq'], np.log10(data0['abstract_freq']))

C:\Users\jxlji\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\jxlji\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\jxlji\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\jxlji\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\jxlji\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [87]:
data0.to_csv("golden_self.tsv",sep="\t",index=False)

In [214]:
data0=pd.read_csv("golden_self.tsv",sep="\t")

In [16]:
unique_list=data0.PMID.unique()

In [18]:
unique_df=pd.DataFrame({"PMID":unique_list})

In [19]:
unique_df

,PMID
0,24390784
1,24393089
2,24421179
3,24494939
4,24577789
...,...
283,26986796
284,26990006
285,27062604
286,27449091


In [20]:
# 'PPUB.tsv.gz'
# http://abel.lis.illinois.edu/data/PubMed-citation-2018/PPUB.tsv.gz
start_time = time.time()
column_names=['fromPMID', 'toPMID']
PPUB_df = pd.read_csv('PPUB.tsv.gz', sep='\t',compression='gzip', usecols=column_names)#,skiprows=skip_rows, nrows=n_rows_to_read)#, names=column_names)
end_time = time.time()
print(f"Processing time：{end_time - start_time} seconds")

# # 仅保留 PPUB_df 中需要的列，以节省内存
# PPUB_df = PPUB_df[['fromPMID', 'toPMID']]

Processing time：343.6031675338745 seconds


In [26]:
# 使用内部合并，仅保留两个数据集中共有的行
result_df = pd.merge(PPUB_df, unique_df[['PMID']], left_on='fromPMID', right_on='PMID', how='inner')

# 删除辅助列
result_df.drop('PMID', axis=1, inplace=True)

In [40]:
unique_cited=result_df.toPMID.unique()

In [42]:
len(unique_cited)

9577

In [43]:
unique_cited_df=pd.DataFrame({"PMID":unique_cited})

In [44]:
unique_cited_df

,PMID
0,1393709
1,2487128
2,6684981
3,7555215
4,8958452
...,...
9572,27118665
9573,27811122
9574,27839818
9575,28317067


In [27]:
# filtered_cited = PPUB_df[PPUB_df['fromPMID'].isin(unique_list)]

In [29]:
toPMIDlist_df = result_df.groupby('fromPMID')['toPMID'].apply(list).reset_index(name='toPMIDlist')

In [ ]:
toPMIDlist_df

In [30]:
toPMIDlist_df

,fromPMID,toPMIDlist
0,24393089,"[1393709, 2487128, 6684981, 7555215, 8958452, ..."
1,24399715,"[2149639, 2530095, 5064045, 7589207, 7613632, ..."
2,24421179,"[11000103, 11841668, 12481133, 12651368, 16043..."
3,24453941,"[196284, 6294151, 7580127, 7931533, 8734579, 9..."
4,24494939,"[1531583, 2021661, 2738307, 3205267, 7560708, ..."
...,...,...
264,30440893,"[8724468, 21624928, 23410655, 23453041, 25014975]"
265,30452750,"[7573622, 12142772, 12171751, 12668754, 162388..."
266,30466303,"[15916493, 17588200, 19532050, 20624319, 21226..."
267,30468343,"[2539074, 2831868, 3014441, 4989111, 8126466, ..."


In [34]:
toPMIDlist_df = toPMIDlist_df.rename(columns={'fromPMID': 'PMID'})
toPMIDlist_df = toPMIDlist_df.rename(columns={'toPMIDlist': 'cited_PMID'})

In [35]:
toPMIDlist_df.to_csv('golden_cited_info.tsv', sep='\t', index=False)

In [13]:
start_time = time.time()
column_names=["PMID","MeSH"]
# http://abel.lis.illinois.edu/data/mesh2018.tsv.gz
mesh_df = pd.read_csv('mesh2018.tsv', sep='\t')#,skiprows=skip_rows, nrows=n_rows_to_read)#, usecols=column_names)
end_time = time.time()
print(f"Processing time：{end_time - start_time} seconds")

Processing time：44.054235219955444 seconds


In [36]:
start_time = time.time()
merged_df = pd.merge(toPMIDlist_df, mesh_df, on='PMID', how='left')
end_time = time.time()
print(f"Processing time：{end_time - start_time} seconds")

Processing time：16.2046799659729 seconds


In [52]:
merged_df = merged_df.rename(columns={'cited_PMID': 'citedPMID'})

In [45]:
start_time = time.time()
unique_cited_df_mesh = pd.merge(unique_cited_df, mesh_df, on='PMID', how='left')
end_time = time.time()
print(f"Processing time：{end_time - start_time} seconds")

Processing time：3.7132046222686768 seconds


In [47]:
unique_cited_df_mesh.to_csv("golden_cited_mesh.tsv",sep="\t",index=False)

In [111]:
unique_cited_df_mesh

,mesh
PMID,
1393709,African Americans|Chicago|Child|Child Abuse|Ch...
2487128,Baltimore|Child|Child Abuse|Cross-Sectional St...
6684981,Child|Child Abuse|Ethnic Groups|Family Charact...
7555215,"Adolescent|Child|Child Abuse|Child, Preschool|..."
8958452,"Adolescent|Child|Child Abuse|Child, Preschool|..."
...,...
27118665,"Adolescent|Adult|Age Distribution|Aged|Aged, 8..."
27811122,Aged|Autonomic Nervous System Diseases|Ephedri...
27839818,Animals|Artifacts|Cardiac Imaging Techniques|F...


In [61]:
# del PPUB
# del mesh_df
# unique_cited_df_mesh.isna().any()

mesh    False
dtype: bool

In [64]:
import pandas as pd
import ast
from tqdm import tqdm
import time

try:
    unique_cited_df_mesh.set_index('PMID', inplace=True)
except: 
    pass

cited_mesh = []

start_time = time.time()

for cited_str in tqdm(merged_df['citedPMID']):
    try:
        cited_list = ast.literal_eval(cited_str)
    except:
        cited_list=cited_str
    current_meshes = set()
    
    for pmid in cited_list:
        mesh_str = unique_cited_df_mesh.loc[pmid, 'mesh'] if pmid in unique_cited_df_mesh.index else ''
        current_meshes.update(mesh_str.split('|'))
    
    cited_mesh.append('|'.join(current_meshes))

end_time = time.time()

elapsed_time = end_time - start_time
print(f"completed spend time in seconds：{elapsed_time}")

100%|██████████████████████████████████████████████████████████████████████████████| 269/269 [00:00<00:00, 2534.68it/s]

匹配完成，耗时：0.11276602745056152秒


In [68]:
merged_df['cited_mesh'] = cited_mesh

# 输出查看
merged_df.head()

,PMID,citedPMID,mesh,cited_mesh
0,24393089,"[1393709, 2487128, 6684981, 7555215, 8958452, ...","Adult|Anomie|Child|Child Abuse, Sexual|Crime V...",Incidence|Poverty|Social Values|Choice Behavio...
1,24399715,"[2149639, 2530095, 5064045, 7589207, 7613632, ...","Animals|Antiparkinson Agents|Behavior, Animal|...",Aminobutyrates|2-Amino-5-phosphonovalerate|Act...
2,24421179,"[11000103, 11841668, 12481133, 12651368, 16043...",Carbon Sequestration|Forests|Indiana|Magnoliop...,"Plant Leaves|Plant Roots|-|Biodegradation, Env..."
3,24453941,"[196284, 6294151, 7580127, 7931533, 8734579, 9...",Animals|Cell Proliferation|GTP-Binding Protein...,GTP-Binding Proteins|Myelin-Oligodendrocyte Gl...
4,24494939,"[1531583, 2021661, 2738307, 3205267, 7560708, ...",Accidental Falls|Aged|Body Mass Index|Female|H...,Finland|Morbidity|China|Reproducibility of Res...


In [69]:
merged_df.to_csv("golden_cited_mesh_citedmesh.tsv",sep="\t")

In [77]:
# MapAffilTempo2018.tsv.gz
# http://abel.lis.illinois.edu/data/MapAffilTempo2018.tsv.gz
start_time = time.time()
column_names=["ID","pos","City","City-temporal"]
mapaffil_df = pd.read_csv('MapAffilTempo2018.tsv.gz', sep='\t', compression='gzip', encoding='utf-8', usecols=column_names)#skiprows=skip_rows, nrows=n_rows_to_read,
end_time = time.time()
print(f"Processing time：{end_time - start_time} seconds")

Processing time：113.4690351486206 seconds


In [78]:
mapaffil_df#.reset_index(drop=True)

,ID,pos,City,City-temporal
0,15398801,1,-,NaN
1,10600040,3,"Wroclaw, Poland","Wroclaw, Poland"
2,11315337,3,"Wroclaw, Poland","Wroclaw, Poland"
3,12820124,2,"Wroclaw, Poland","Wroclaw, Poland"
4,14556494,3,"Wroclaw, Poland","Wroclaw, Poland"
...,...,...,...,...
119966685,22398613,46,"Hinxton, Cambridge, Cambridgeshire, UK","Houston, TX, USA"
119966686,25423365,89,"Houston, TX, USA","Houston, TX, USA"
119966687,25660540,53,"Houston, TX, USA","Houston, TX, USA"
119966688,25908251,120,"Houston, TX, USA","Houston, TX, USA"


In [80]:
unique_df["PMID"]

0      24390784
1      24393089
2      24421179
3      24494939
4      24577789
         ...   
283    26986796
284    26990006
285    27062604
286    27449091
287    27451861
Name: PMID, Length: 288, dtype: int64

In [79]:
unique_cited_df["PMID"]

0        1393709
1        2487128
2        6684981
3        7555215
4        8958452
          ...   
9572    27118665
9573    27811122
9574    27839818
9575    28317067
9576    29411169
Name: PMID, Length: 9577, dtype: int64

In [81]:
filtered_df = mapaffil_df[mapaffil_df['ID'].astype(str).isin(unique_df["PMID"].astype(str))]
filtered_df

,ID,pos,City,City-temporal
258572,30514933,9,"Würzburg, Germany","Würzburg, Germany"
288670,25367471,2,"Davos, Switzerland","Davos, Switzerland"
374365,25889511,4,"Yaoundé, Centre, Cameroon","Montpellier, France"
472137,25265236,4,"Darmstadt, Germany","Darmstadt, Germany"
605054,27449091,9,"Los Angeles, CA, USA","Tokyo, Japan"
...,...,...,...,...
119664590,25167450,3,"Nanjing, China","Nanjing, China"
119687830,28810704,1,"Krc, Praha, Hlavní m sto Praha, Czech Republic","Trebon, Jihoceský, Czech Republic"
119703002,25711284,8,"Cheras, Kuala Lumpur, Wilayah Persekutuan, Mal...","Kuala Lumpur, Wilayah Persekutuan, Malaysia"
119728525,25287853,5,"Rostock, Germany","Rostock, Germany"


In [83]:
filtered_df.rename(columns={'ID': 'PMID'}, inplace=True)
filtered_df_list=filtered_df.groupby('PMID').agg(lambda x:list(x))
filtered_df_list=filtered_df_list.reset_index()
filtered_df_list

C:\Users\jxlji\AppData\Local\Temp\ipykernel_19928\2187578585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns={'ID': 'PMID'}, inplace=True)


,PMID,pos,City,City-temporal
0,24390784,[1],"[Rochester, NY, USA]","[Rochester, NY, USA]"
1,24393089,"[2, 4, 3, 1]","[Orlando, FL, USA, Orlando, FL, USA, Orlando, ...","[Louisville, KY, USA, Orlando, FL, USA, Orland..."
2,24399715,"[2, 1]","[Krakow, Poland, Krakow, Poland]","[Krakow, Poland, Krakow, Poland]"
3,24421179,"[8, 6, 5, 1, 7, 2, 4, 3]","[Bloomington, IN, USA, Bloomington, IN, USA, B...","[Bloomington, IN, USA, Newtown Square, PA, USA..."
4,24453941,"[17, 13, 12, 16, 4, 3, 11, 10, 2, 6, 15, 5, 9,...","[Zurich, Switzerland, Muttenz, Basel, Switzerl...","[Zurich, Switzerland, Muttenz, Basel, Switzerl..."
...,...,...,...,...
281,30452750,"[2, 3, 1]","[Tucson, AZ, USA, Tucson, AZ, USA, Tucson, AZ,...","[Tucson, AZ, USA, Tucson, AZ, USA, Tucson, AZ,..."
282,30466303,"[19, 18, 7, 2, 16, 10, 9, 13, 14, 15, 5, 8, 6,...","[Vancouver, BC, Canada, Vancouver, BC, Canada,...","[Vancouver, BC, Canada, Vancouver, BC, Canada,..."
283,30468343,"[2, 1]","[Zagazig, Ash Sharqiyah, Egypt, Zagazig, Ash S...","[Zagazig, Ash Sharqiyah, Egypt, Zagazig, Ash S..."
284,30500121,"[2, 7, 6, 3, 4, 1, 5, 8]","[Rotterdam, Zuid-Holland, Netherlands, Rotterd...","[Rotterdam, Zuid-Holland, Netherlands, Rotterd..."


In [84]:
filtered_df_list['unique_city'] = filtered_df_list['City'].apply(lambda x: ','.join(sorted(set(x))))

In [85]:
filtered_df_list['unique_tempocity'] = filtered_df_list['City-temporal'].apply(
    lambda x: ','.join(
        sorted(
            set(str(item) for item in x)
        )
    )
)

In [88]:
filtered_df_list[["PMID","unique_city","unique_tempocity"]]

,PMID,unique_city,unique_tempocity
0,24390784,"Rochester, NY, USA","Rochester, NY, USA"
1,24393089,"Orlando, FL, USA","Louisville, KY, USA,Orlando, FL, USA"
2,24399715,"Krakow, Poland","Krakow, Poland"
3,24421179,"Bloomington, IN, USA","Bloomington, IN, USA,Garmisch-Partenkirchen, G..."
4,24453941,"Braunschweig, Germany,Muttenz, Basel, Switzerl...","Braunschweig, Germany,Dallas, TX, USA,Heidelbe..."
...,...,...,...
281,30452750,"Tucson, AZ, USA","Tucson, AZ, USA"
282,30466303,"Burnaby, BC, Canada,Kelowna, BC, Canada,New Yo...","Burnaby, BC, Canada,New York, NY, USA,San Dieg..."
283,30468343,"Zagazig, Ash Sharqiyah, Egypt","Zagazig, Ash Sharqiyah, Egypt"
284,30500121,"Rotterdam, Zuid-Holland, Netherlands","Rotterdam, Zuid-Holland, Netherlands"


In [89]:
filtered_df_list[["PMID","unique_city","unique_tempocity"]].to_csv("golden_tempo.tsv",sep="\t",index=False)

In [90]:
filtered_df = mapaffil_df[mapaffil_df['ID'].astype(str).isin(unique_cited_df["PMID"].astype(str))]
filtered_df

,ID,pos,City,City-temporal
418,10573533,7,"Matsumoto, Nagano, Japan","Fukuoka, Japan"
501,18308209,1,"Fukuoka, Japan","Fukuoka, Japan"
1520,20036486,5,"Indianapolis, IN, USA","Indianapolis, IN, USA"
1521,20036488,2,"Indianapolis, IN, USA","Indianapolis, IN, USA"
4895,9641683,45,"Jacksonville, FL, USA","Rotterdam, Zuid-Holland, Netherlands"
...,...,...,...,...
119946042,26972586,4,"Xiamen, China","Xiamen, China"
119947572,25879705,1,"Addis Ababa, Oromia, Ethiopia","Addis Ababa, Oromia, Ethiopia"
119952760,15231968,6,"Atlanta, GA, USA","Atlanta, GA, USA"
119954499,27846951,3,"Los Angeles, CA, USA","Los Angeles, CA, USA"


In [91]:
filtered_df.rename(columns={'ID': 'PMID'}, inplace=True)
filtered_df_list=filtered_df.groupby('PMID').agg(lambda x:list(x))
filtered_df_list=filtered_df_list.reset_index()
filtered_df_list

C:\Users\jxlji\AppData\Local\Temp\ipykernel_19928\2187578585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns={'ID': 'PMID'}, inplace=True)


,PMID,pos,City,City-temporal
0,10003356,"[4, 1, 5, 3, 2]","[London, UK, London, UK, London, UK, London, U...","[London, UK, London, UK, London, UK, London, U..."
1,10004425,"[1, 3, 2]","[Warren, MI, USA, Columbia, MO, USA, Warren, M...","[Warren, MI, USA, Columbia, MO, USA, Warren, M..."
2,10021465,"[5, 2, 1, 3, 4]","[Cincinnati, OH, USA, Cincinnati, OH, USA, Cin...","[Cincinnati, OH, USA, Cincinnati, OH, USA, Cin..."
3,10022404,"[2, 4, 5, 1, 3]","[Bethesda, MD, USA, Bethesda, MD, USA, Bethesd...","[Bethesda, MD, USA, Bethesda, MD, USA, Baltimo..."
4,10022830,"[4, 5, 3, 2, 1]","[Philadelphia, PA, USA, Philadelphia, PA, USA,...","[Philadelphia, PA, USA, Philadelphia, PA, USA,..."
...,...,...,...,...
9540,9946999,[1],"[Johannesburg, Gauteng, South Africa]","[Johannesburg, Gauteng, South Africa]"
9541,9950829,"[2, 3, 4, 1]","[New Haven, CT, USA, New Haven, CT, USA, New H...","[New Haven, CT, USA, New Haven, CT, USA, New H..."
9542,9977254,"[1, 2]","[Newcastle Upon Tyne, Tyne and Wear, UK, Newca...","[Newcastle Upon Tyne, Tyne and Wear, UK, Newca..."
9543,9987176,"[2, 5, 4, 1, 6, 3]","[Taipei, Taiwan, Taipei, Taiwan, Taipei, Taiwa...","[Beitou, Taipei, Taiwan, Beitou, Taipei, Taiwa..."


In [92]:
filtered_df_list['unique_city'] = filtered_df_list['City'].apply(lambda x: ','.join(sorted(set(x))))
filtered_df_list['unique_tempocity'] = filtered_df_list['City-temporal'].apply(
    lambda x: ','.join(
        sorted(
            set(str(item) for item in x)
        )
    )
)
filtered_df_list[["PMID","unique_city","unique_tempocity"]]

,PMID,unique_city,unique_tempocity
0,10003356,"London, UK","London, UK"
1,10004425,"Columbia, MO, USA,Warren, MI, USA","Columbia, MO, USA,Warren, MI, USA"
2,10021465,"Cincinnati, OH, USA","Cincinnati, OH, USA"
3,10022404,"Bethesda, MD, USA","Baltimore, MD, USA,Bethesda, MD, USA"
4,10022830,"Philadelphia, PA, USA","Dallas, TX, USA,Philadelphia, PA, USA"
...,...,...,...
9540,9946999,"Johannesburg, Gauteng, South Africa","Johannesburg, Gauteng, South Africa"
9541,9950829,"New Haven, CT, USA","New Haven, CT, USA"
9542,9977254,"Newcastle Upon Tyne, Tyne and Wear, UK","Newcastle Upon Tyne, Tyne and Wear, UK"
9543,9987176,"Taipei, Taiwan","Beitou, Taipei, Taiwan,Taipei, Taiwan"


In [93]:
filtered_df_list[["PMID","unique_city","unique_tempocity"]].to_csv("golden_cited_tempo.tsv",sep="\t",index=False)

In [97]:
merged_df

,PMID,citedPMID,mesh,cited_mesh
0,24393089,"[1393709, 2487128, 6684981, 7555215, 8958452, ...","Adult|Anomie|Child|Child Abuse, Sexual|Crime V...",Incidence|Poverty|Social Values|Choice Behavio...
1,24399715,"[2149639, 2530095, 5064045, 7589207, 7613632, ...","Animals|Antiparkinson Agents|Behavior, Animal|...",Aminobutyrates|2-Amino-5-phosphonovalerate|Act...
2,24421179,"[11000103, 11841668, 12481133, 12651368, 16043...",Carbon Sequestration|Forests|Indiana|Magnoliop...,"Plant Leaves|Plant Roots|-|Biodegradation, Env..."
3,24453941,"[196284, 6294151, 7580127, 7931533, 8734579, 9...",Animals|Cell Proliferation|GTP-Binding Protein...,GTP-Binding Proteins|Myelin-Oligodendrocyte Gl...
4,24494939,"[1531583, 2021661, 2738307, 3205267, 7560708, ...",Accidental Falls|Aged|Body Mass Index|Female|H...,Finland|Morbidity|China|Reproducibility of Res...
...,...,...,...,...
264,30440893,"[8724468, 21624928, 23410655, 23453041, 25014975]",-,Neurologic Examination|Age Factors|Artificial ...
265,30452750,"[7573622, 12142772, 12171751, 12668754, 162388...",-,Health Planning|Bioterrorism|Morbidity|Smartph...
266,30466303,"[15916493, 17588200, 19532050, 20624319, 21226...",-,Acquired Immunodeficiency Syndrome|Adolescent|...
267,30468343,"[2539074, 2831868, 3014441, 4989111, 8126466, ...",-,"DNA, Complementary|Software|Internet|Cell Cult..."


In [232]:
tempo_df=pd.read_csv("golden_tempo.tsv",sep="\t")

In [233]:
tempo_df

,PMID,unique_city,unique_tempocity
0,24390784,"Rochester, NY, USA","Rochester, NY, USA"
1,24393089,"Orlando, FL, USA","Louisville, KY, USA,Orlando, FL, USA"
2,24399715,"Krakow, Poland","Krakow, Poland"
3,24421179,"Bloomington, IN, USA","Bloomington, IN, USA,Garmisch-Partenkirchen, G..."
4,24453941,"Braunschweig, Germany,Muttenz, Basel, Switzerl...","Braunschweig, Germany,Dallas, TX, USA,Heidelbe..."
...,...,...,...
281,30452750,"Tucson, AZ, USA","Tucson, AZ, USA"
282,30466303,"Burnaby, BC, Canada,Kelowna, BC, Canada,New Yo...","Burnaby, BC, Canada,New York, NY, USA,San Dieg..."
283,30468343,"Zagazig, Ash Sharqiyah, Egypt","Zagazig, Ash Sharqiyah, Egypt"
284,30500121,"Rotterdam, Zuid-Holland, Netherlands","Rotterdam, Zuid-Holland, Netherlands"


In [234]:
start_time = time.time()
merged_df = pd.merge(merged_df, tempo_df, on='PMID', how='left')
end_time = time.time()
print(f"Processing time：{end_time - start_time} seconds")

Processing time：0.0014581680297851562 seconds


In [235]:
merged_df=merged_df.drop('unique_city', axis=1)

In [237]:
merged_df

,PMID,cited_PMID,mesh,unique_city_x,unique_city_y,unique_tempocity
0,24393089,"[1393709, 2487128, 6684981, 7555215, 8958452, ...","Adult|Anomie|Child|Child Abuse, Sexual|Crime V...","Orlando, FL, USA","Orlando, FL, USA","Louisville, KY, USA,Orlando, FL, USA"
1,24399715,"[2149639, 2530095, 5064045, 7589207, 7613632, ...","Animals|Antiparkinson Agents|Behavior, Animal|...","Krakow, Poland","Krakow, Poland","Krakow, Poland"
2,24421179,"[11000103, 11841668, 12481133, 12651368, 16043...",Carbon Sequestration|Forests|Indiana|Magnoliop...,"Bloomington, IN, USA","Bloomington, IN, USA","Bloomington, IN, USA,Garmisch-Partenkirchen, G..."
3,24453941,"[196284, 6294151, 7580127, 7931533, 8734579, 9...",Animals|Cell Proliferation|GTP-Binding Protein...,"Braunschweig, Germany,Muttenz, Basel, Switzerl...","Braunschweig, Germany,Muttenz, Basel, Switzerl...","Braunschweig, Germany,Dallas, TX, USA,Heidelbe..."
4,24494939,"[1531583, 2021661, 2738307, 3205267, 7560708, ...",Accidental Falls|Aged|Body Mass Index|Female|H...,"NSW, Australia","NSW, Australia","Sydney, NSW, Australia"
...,...,...,...,...,...,...
264,30440893,"[8724468, 21624928, 23410655, 23453041, 25014975]",-,-,-,"Jinan, Shandong, China,Zhengzhou, China,nan"
265,30452750,"[7573622, 12142772, 12171751, 12668754, 162388...",-,"Tucson, AZ, USA","Tucson, AZ, USA","Tucson, AZ, USA"
266,30466303,"[15916493, 17588200, 19532050, 20624319, 21226...",-,"Burnaby, BC, Canada,Kelowna, BC, Canada,New Yo...","Burnaby, BC, Canada,Kelowna, BC, Canada,New Yo...","Burnaby, BC, Canada,New York, NY, USA,San Dieg..."
267,30468343,"[2539074, 2831868, 3014441, 4989111, 8126466, ...",-,"Zagazig, Ash Sharqiyah, Egypt","Zagazig, Ash Sharqiyah, Egypt","Zagazig, Ash Sharqiyah, Egypt"


In [238]:
cited_tempo_df=pd.read_csv("golden_cited_tempo.tsv",sep="\t")

In [239]:
cited_mesh_df=pd.read_csv("golden_cited_mesh.tsv",sep="\t")

In [240]:
start_time = time.time()
merged = pd.merge(cited_tempo_df, cited_mesh_df, on='PMID', how='left')
end_time = time.time()
print(f"Processing time：{end_time - start_time} seconds")

Processing time：0.004562854766845703 seconds


In [241]:
merged

,PMID,unique_city,unique_tempocity,mesh
0,10003356,"London, UK","London, UK",-
1,10004425,"Columbia, MO, USA,Warren, MI, USA","Columbia, MO, USA,Warren, MI, USA",-
2,10021465,"Cincinnati, OH, USA","Cincinnati, OH, USA","Administration, Inhalation|Animals|Bronchoalve..."
3,10022404,"Bethesda, MD, USA","Baltimore, MD, USA,Bethesda, MD, USA","Adenoma|Adolescent|Adult|Aged|Aged, 80 and ove..."
4,10022830,"Philadelphia, PA, USA","Dallas, TX, USA,Philadelphia, PA, USA",Animals|Arrestins|Binding Sites|COS Cells|Catt...
...,...,...,...,...
9540,9946999,"Johannesburg, Gauteng, South Africa","Johannesburg, Gauteng, South Africa",-
9541,9950829,"New Haven, CT, USA","New Haven, CT, USA","Animals|Blotting, Northern|Blotting, Western|C..."
9542,9977254,"Newcastle Upon Tyne, Tyne and Wear, UK","Newcastle Upon Tyne, Tyne and Wear, UK",-
9543,9987176,"Taipei, Taiwan","Beitou, Taipei, Taiwan,Taipei, Taiwan",Child|Follow-Up Studies|Hepatitis B Surface An...


In [246]:
merged_df.rename(columns={'unique_city_x': 'unique_city'}, inplace=True)
merged_df=merged_df.drop('unique_city_y', axis=1)

In [247]:
merged_df

,PMID,cited_PMID,mesh,unique_city,unique_tempocity
0,24393089,"[1393709, 2487128, 6684981, 7555215, 8958452, ...","Adult|Anomie|Child|Child Abuse, Sexual|Crime V...","Orlando, FL, USA","Louisville, KY, USA,Orlando, FL, USA"
1,24399715,"[2149639, 2530095, 5064045, 7589207, 7613632, ...","Animals|Antiparkinson Agents|Behavior, Animal|...","Krakow, Poland","Krakow, Poland"
2,24421179,"[11000103, 11841668, 12481133, 12651368, 16043...",Carbon Sequestration|Forests|Indiana|Magnoliop...,"Bloomington, IN, USA","Bloomington, IN, USA,Garmisch-Partenkirchen, G..."
3,24453941,"[196284, 6294151, 7580127, 7931533, 8734579, 9...",Animals|Cell Proliferation|GTP-Binding Protein...,"Braunschweig, Germany,Muttenz, Basel, Switzerl...","Braunschweig, Germany,Dallas, TX, USA,Heidelbe..."
4,24494939,"[1531583, 2021661, 2738307, 3205267, 7560708, ...",Accidental Falls|Aged|Body Mass Index|Female|H...,"NSW, Australia","Sydney, NSW, Australia"
...,...,...,...,...,...
264,30440893,"[8724468, 21624928, 23410655, 23453041, 25014975]",-,-,"Jinan, Shandong, China,Zhengzhou, China,nan"
265,30452750,"[7573622, 12142772, 12171751, 12668754, 162388...",-,"Tucson, AZ, USA","Tucson, AZ, USA"
266,30466303,"[15916493, 17588200, 19532050, 20624319, 21226...",-,"Burnaby, BC, Canada,Kelowna, BC, Canada,New Yo...","Burnaby, BC, Canada,New York, NY, USA,San Dieg..."
267,30468343,"[2539074, 2831868, 3014441, 4989111, 8126466, ...",-,"Zagazig, Ash Sharqiyah, Egypt","Zagazig, Ash Sharqiyah, Egypt"


In [132]:
merged_df.rename(columns={'citedPMID': 'cited_PMID'}, inplace=True)

In [248]:
def process_row(row):
    new_row = row.copy()
    cited_info = {f'cited_{col}': [] for col in merged.columns if col != 'PMID'}

    try:
        cited_pmids = ast.literal_eval(row['cited_PMID'])
    except:
        cited_pmids = row['cited_PMID']

    for cited_pmid in cited_pmids:
        matched_rows = merged[merged['PMID'] == cited_pmid]

        if not matched_rows.empty:
            for col in merged.columns:

                if col != 'PMID':
                    cited_info[f'cited_{col}'].extend(matched_rows[col].tolist())
        else:
            for col in merged.columns:
                if col != 'PMID':
                    cited_info[f'cited_{col}'].append([])

    for col, values in cited_info.items():
        unique_cities_set = set()
        for item in values:
            if isinstance(item, str):
                items = [i.strip() for i in item.split(',') if col != 'cited_mesh']
                items += [i.strip() for i in item.split('|') if col == 'cited_mesh']  
                unique_cities_set.update(items)
            elif isinstance(item, list) and item:  
                cities = [city.strip() for city in item]
                unique_cities_set.update(item)

        if '-' in unique_cities_set:
            unique_cities_set.remove('-')

        unique_cities_list = list(unique_cities_set)
        new_row[col] = unique_cities_list

    return pd.Series(new_row)

result_columns = list(merged_df.columns) + [f'cited_{col}' for col in merged.columns if col != 'PMID']
result_df = pd.DataFrame(columns=result_columns)

batch_size = 2000
# if data is large
for start in range(0, 1600000, batch_size):
    print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    end = start + batch_size
    print(f"Processing rows {start} to {end} at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    batch_df = merged_df.iloc[start:end].apply(process_row, axis=1)
    result_df = pd.concat([result_df, batch_df], ignore_index=True)

print(f"Processing complete at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

2024-05-02 12:36:31
Processing rows 0 to 2000 at 2024-05-02 12:36:31
2024-05-02 12:36:32
Processing rows 2000 to 4000 at 2024-05-02 12:36:32
2024-05-02 12:36:32
Processing rows 4000 to 6000 at 2024-05-02 12:36:32
2024-05-02 12:36:32
Processing rows 6000 to 8000 at 2024-05-02 12:36:32
2024-05-02 12:36:32
Processing rows 8000 to 10000 at 2024-05-02 12:36:32
2024-05-02 12:36:32
Processing rows 10000 to 12000 at 2024-05-02 12:36:32
2024-05-02 12:36:32
Processing rows 12000 to 14000 at 2024-05-02 12:36:32
2024-05-02 12:36:32
Processing rows 14000 to 16000 at 2024-05-02 12:36:32
2024-05-02 12:36:32
Processing rows 16000 to 18000 at 2024-05-02 12:36:32
2024-05-02 12:36:32
Processing rows 18000 to 20000 at 2024-05-02 12:36:32
2024-05-02 12:36:32
Processing rows 20000 to 22000 at 2024-05-02 12:36:32
2024-05-02 12:36:32
Processing rows 22000 to 24000 at 2024-05-02 12:36:32
2024-05-02 12:36:32
Processing rows 24000 to 26000 at 2024-05-02 12:36:32
2024-05-02 12:36:32
Processing rows 26000 to 28000

2024-05-02 12:36:33
Processing rows 866000 to 868000 at 2024-05-02 12:36:33
2024-05-02 12:36:33
Processing rows 868000 to 870000 at 2024-05-02 12:36:33
2024-05-02 12:36:33
Processing rows 870000 to 872000 at 2024-05-02 12:36:33
2024-05-02 12:36:33
Processing rows 872000 to 874000 at 2024-05-02 12:36:33
2024-05-02 12:36:33
Processing rows 874000 to 876000 at 2024-05-02 12:36:33
2024-05-02 12:36:33
Processing rows 876000 to 878000 at 2024-05-02 12:36:33
2024-05-02 12:36:33
Processing rows 878000 to 880000 at 2024-05-02 12:36:33
2024-05-02 12:36:33
Processing rows 880000 to 882000 at 2024-05-02 12:36:33
2024-05-02 12:36:33
Processing rows 882000 to 884000 at 2024-05-02 12:36:33
2024-05-02 12:36:33
Processing rows 884000 to 886000 at 2024-05-02 12:36:33
2024-05-02 12:36:33
Processing rows 886000 to 888000 at 2024-05-02 12:36:33
2024-05-02 12:36:33
Processing rows 888000 to 890000 at 2024-05-02 12:36:33
2024-05-02 12:36:33
Processing rows 890000 to 892000 at 2024-05-02 12:36:33
2024-05-02 1

In [249]:
result_df.to_csv("golden_cited_all.tsv",sep="\t",index=False)

In [250]:
result_df

,PMID,cited_PMID,mesh,unique_city,unique_tempocity,cited_unique_city,cited_unique_tempocity,cited_mesh
0,24393089,"[1393709, 2487128, 6684981, 7555215, 8958452, ...","Adult|Anomie|Child|Child Abuse, Sexual|Crime V...","Orlando, FL, USA","Louisville, KY, USA,Orlando, FL, USA","[OK, WA, Tacoma, Los Angeles, Baltimore, Cleve...","[Seattle, TX, Charleston, Austin, Los Angeles,...","[Incidence, Poverty, Social Values, Choice Beh..."
1,24399715,"[2149639, 2530095, 5064045, 7589207, 7613632, ...","Animals|Antiparkinson Agents|Behavior, Animal|...","Krakow, Poland","Krakow, Poland","[Yantai, London, China, Indianapolis, TN, Krak...","[Yantai, Aachen, London, China, Indianapolis, ...","[Aminobutyrates, 2-Amino-5-phosphonovalerate, ..."
2,24421179,"[11000103, 11841668, 12481133, 12651368, 16043...",Carbon Sequestration|Forests|Indiana|Magnoliop...,"Bloomington, IN, USA","Bloomington, IN, USA,Garmisch-Partenkirchen, G...","[Fort Wainwright, Finland, AZ, China, Tucson, ...","[Fort Wainwright, Finland, AZ, KS, China, Tucs...","[Plant Leaves, Plant Roots, Biodegradation, En..."
3,24453941,"[196284, 6294151, 7580127, 7931533, 8734579, 9...",Animals|Cell Proliferation|GTP-Binding Protein...,"Braunschweig, Germany,Muttenz, Basel, Switzerl...","Braunschweig, Germany,Dallas, TX, USA,Heidelbe...","[Ishikawa, Suita, CT, DC, USA, Oxford, Kanazaw...","[China, Ishikawa, Suita, CT, East Hanover, DC,...","[GTP-Binding Proteins, Myelin-Oligodendrocyte ..."
4,24494939,"[1531583, 2021661, 2738307, 3205267, 7560708, ...",Accidental Falls|Aged|Body Mass Index|Female|H...,"NSW, Australia","Sydney, NSW, Australia","[Finland, Oklahoma City, China, TN, NY, Westwo...","[Queenstown, Finland, London, Oklahoma City, C...","[Finland, Morbidity, China, Reproducibility of..."
...,...,...,...,...,...,...,...,...
264,30440893,"[8724468, 21624928, 23410655, 23453041, 25014975]",-,-,"Jinan, Shandong, China,Zhengzhou, China,nan","[Shanghai, China, Louisville, Southampton, Cle...","[Shanghai, China, Louisville, Southampton, Cle...","[Neurologic Examination, Age Factors, Artifici..."
265,30452750,"[7573622, 12142772, 12171751, 12668754, 162388...",-,"Tucson, AZ, USA","Tucson, AZ, USA","[Denver, Seattle, London, AZ, Afula, Tucson, P...","[Denver, Seattle, AZ, Tucson, SD, DC, Los Ange...","[Health Planning, Bioterrorism, Morbidity, Sma..."
266,30466303,"[15916493, 17588200, 19532050, 20624319, 21226...",-,"Burnaby, BC, Canada,Kelowna, BC, Canada,New Yo...","Burnaby, BC, Canada,New York, NY, USA,San Dieg...","[Seattle, Boulogne-Billancourt, Seoul, Anderle...","[Seattle, Boulogne-Billancourt, Seoul, Anderle...","[Acquired Immunodeficiency Syndrome, Adolescen..."
267,30468343,"[2539074, 2831868, 3014441, 4989111, 8126466, ...",-,"Zagazig, Ash Sharqiyah, Egypt","Zagazig, Ash Sharqiyah, Egypt","[AZ, Oyo State, Hachioji, Bellaterra, College ...","[Trinidad and Tobago, AZ, Oyo State, Hachioji,...","[DNA, Complementary, Software, Internet, Cell ..."


In [251]:
mesh_data=pd.read_excel("Merged_res.xlsx")
mesh_data=mesh_data.drop(['Unnamed: 0'], axis=1)
mesh_data

,Category,Name,Co-occur,Total Articles,Rate,Level,Merged
0,I,"Anthropology, Education, Sociology, and Social...",1516538,2954827,0.513241,A,I
1,K,Humanities,434315,849376,0.511334,A,K
2,N,Health Care,3313445,10388431,0.318955,B,N
3,F,Psychiatry and Psychology,1176954,4246883,0.277134,B,F
4,H,Disciplines and Occupations,1007407,3646994,0.276229,B,H
...,...,...,...,...,...,...,...
57968,B01.050.150.900.649.313.992.635.505.500.800.50...,"Mice, Knockout, ApoE",1,143,0.006993,D,B01.050.150.900.649.313.992.635.505
57969,B01.050.500.131.617.720.500.500.750.712.500.87...,Aedes,5495,13739,0.399956,B,B01.050.500.131.617.720.500.500.750.712.500.87...
57970,B01.050.500.131.617.720.500.500.750.712.500.87...,Anopheles,6384,12759,0.500353,A,B01.050.500.131.617.720.500.500.750.712.500.87...
57971,B01.050.500.131.617.720.500.500.750.712.500.87...,Culex,3465,7199,0.481317,B,B01.050.500.131.617.720.500.500.750.712.500.87...


In [252]:
mesh_dict = {row['Name']: [row['Co-occur'], row['Total Articles'], row['Rate']] for index, row in mesh_data.iterrows()}
mesh_dict

{'Anthropology, Education, Sociology, and Social Phenomena': [1516538,
  2954827,
  0.5132408767078411],
 'Humanities': [434315, 849376, 0.5113342029913724],
 'Health Care': [3313445, 10388431, 0.3189552878581953],
 'Psychiatry and Psychology': [1176954, 4246883, 0.2771336059882036],
 'Disciplines and Occupations': [1007407, 3646994, 0.2762294097549927],
 'Technology, Industry, and Agriculture': [388202,
  1769328,
  0.2194064639230262],
 'Named Groups': [2046223, 9177176, 0.2229686997394405],
 'Information Science': [590771, 2847399, 0.2074774206214162],
 'Organisms': [3369483, 23153025, 0.1455310051278397],
 'Analytical, Diagnostic and Therapeutic Techniques, and Equipment': [2502559,
  17403090,
  0.1437996930430171],
 'Diseases': [1909422, 13948484, 0.1368910055028202],
 'Phenomena and Processes': [1429083, 13886302, 0.1029131441905844],
 'Chemicals and Drugs': [1060634, 13691898, 0.07746435154570973],
 'Anatomy': [16421, 385092, 0.04264175833307365],
 'Body Regions': [40211, 77969

In [253]:
def get_top_three_rates(mesh_terms_str):
    mesh_terms = mesh_terms_str.split('|')
    rates = [mesh_dict.get(term, [None, None, None])[2] for term in mesh_terms if term in mesh_dict]
    top_three_rates = sorted(filter(None, rates), reverse=True)[:3]
    last_three_rates = sorted(filter(None, rates), reverse=False)[:3]
    top_three_rates += [0] * (3 - len(top_three_rates))
    last_three_rates += [0] * (3 - len(last_three_rates))
    return top_three_rates+last_three_rates

top_rates_df = result_df['mesh'].apply(get_top_three_rates).apply(pd.Series)
top_rates_df.columns = ['mesh_rate_top1', 'mesh_rate_top2', 'mesh_rate_top3','mesh_rate_last1', 'mesh_rate_last2', 'mesh_rate_last3']

top_rates_df['PMID'] = result_df['PMID']
top_rates_df.set_index('PMID', inplace=True)

In [254]:
top_rates_df['PMID'] = result_df['PMID']
top_rates_df.reset_index(drop=True, inplace=True)
top_rates_df

,mesh_rate_top1,mesh_rate_top2,mesh_rate_top3,mesh_rate_last1,mesh_rate_last2,mesh_rate_last3,PMID
0,0.531818,0.525750,0.363305,0.173560,0.180219,0.191754,NaN
1,0.185198,0.149210,0.148166,0.001764,0.006196,0.006313,NaN
2,0.721929,0.535286,0.498188,0.038730,0.107591,0.132066,NaN
3,0.149210,0.037437,0.022442,0.001632,0.002277,0.002976,NaN
4,0.714289,0.671209,0.509773,0.173560,0.213041,0.260264,NaN
...,...,...,...,...,...,...,...
264,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
265,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
266,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
267,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN


In [255]:
result_df["mesh_rate_top1"]=top_rates_df["mesh_rate_top1"]
result_df["mesh_rate_top2"]=top_rates_df["mesh_rate_top2"]
result_df["mesh_rate_top3"]=top_rates_df["mesh_rate_top3"]
result_df["mesh_rate_last1"]=top_rates_df["mesh_rate_last1"]
result_df["mesh_rate_last2"]=top_rates_df["mesh_rate_last2"]
result_df["mesh_rate_last3"]=top_rates_df["mesh_rate_last3"]

In [256]:
result_df

,PMID,cited_PMID,mesh,unique_city,unique_tempocity,cited_unique_city,cited_unique_tempocity,cited_mesh,mesh_rate_top1,mesh_rate_top2,mesh_rate_top3,mesh_rate_last1,mesh_rate_last2,mesh_rate_last3
0,24393089,"[1393709, 2487128, 6684981, 7555215, 8958452, ...","Adult|Anomie|Child|Child Abuse, Sexual|Crime V...","Orlando, FL, USA","Louisville, KY, USA,Orlando, FL, USA","[OK, WA, Tacoma, Los Angeles, Baltimore, Cleve...","[Seattle, TX, Charleston, Austin, Los Angeles,...","[Incidence, Poverty, Social Values, Choice Beh...",0.531818,0.525750,0.363305,0.173560,0.180219,0.191754
1,24399715,"[2149639, 2530095, 5064045, 7589207, 7613632, ...","Animals|Antiparkinson Agents|Behavior, Animal|...","Krakow, Poland","Krakow, Poland","[Yantai, London, China, Indianapolis, TN, Krak...","[Yantai, Aachen, London, China, Indianapolis, ...","[Aminobutyrates, 2-Amino-5-phosphonovalerate, ...",0.185198,0.149210,0.148166,0.001764,0.006196,0.006313
2,24421179,"[11000103, 11841668, 12481133, 12651368, 16043...",Carbon Sequestration|Forests|Indiana|Magnoliop...,"Bloomington, IN, USA","Bloomington, IN, USA,Garmisch-Partenkirchen, G...","[Fort Wainwright, Finland, AZ, China, Tucson, ...","[Fort Wainwright, Finland, AZ, KS, China, Tucs...","[Plant Leaves, Plant Roots, Biodegradation, En...",0.721929,0.535286,0.498188,0.038730,0.107591,0.132066
3,24453941,"[196284, 6294151, 7580127, 7931533, 8734579, 9...",Animals|Cell Proliferation|GTP-Binding Protein...,"Braunschweig, Germany,Muttenz, Basel, Switzerl...","Braunschweig, Germany,Dallas, TX, USA,Heidelbe...","[Ishikawa, Suita, CT, DC, USA, Oxford, Kanazaw...","[China, Ishikawa, Suita, CT, East Hanover, DC,...","[GTP-Binding Proteins, Myelin-Oligodendrocyte ...",0.149210,0.037437,0.022442,0.001632,0.002277,0.002976
4,24494939,"[1531583, 2021661, 2738307, 3205267, 7560708, ...",Accidental Falls|Aged|Body Mass Index|Female|H...,"NSW, Australia","Sydney, NSW, Australia","[Finland, Oklahoma City, China, TN, NY, Westwo...","[Queenstown, Finland, London, Oklahoma City, C...","[Finland, Morbidity, China, Reproducibility of...",0.714289,0.671209,0.509773,0.173560,0.213041,0.260264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,30440893,"[8724468, 21624928, 23410655, 23453041, 25014975]",-,-,"Jinan, Shandong, China,Zhengzhou, China,nan","[Shanghai, China, Louisville, Southampton, Cle...","[Shanghai, China, Louisville, Southampton, Cle...","[Neurologic Examination, Age Factors, Artifici...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
265,30452750,"[7573622, 12142772, 12171751, 12668754, 162388...",-,"Tucson, AZ, USA","Tucson, AZ, USA","[Denver, Seattle, London, AZ, Afula, Tucson, P...","[Denver, Seattle, AZ, Tucson, SD, DC, Los Ange...","[Health Planning, Bioterrorism, Morbidity, Sma...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
266,30466303,"[15916493, 17588200, 19532050, 20624319, 21226...",-,"Burnaby, BC, Canada,Kelowna, BC, Canada,New Yo...","Burnaby, BC, Canada,New York, NY, USA,San Dieg...","[Seattle, Boulogne-Billancourt, Seoul, Anderle...","[Seattle, Boulogne-Billancourt, Seoul, Anderle...","[Acquired Immunodeficiency Syndrome, Adolescen...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
267,30468343,"[2539074, 2831868, 3014441, 4989111, 8126466, ...",-,"Zagazig, Ash Sharqiyah, Egypt","Zagazig, Ash Sharqiyah, Egypt","[AZ, Oyo State, Hachioji, Bellaterra, College ...","[Trinidad and Tobago, AZ, Oyo State, Hachioji,...","[DNA, Complementary, Software, Internet, Cell ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [257]:
def get_top_three_rates(mesh_terms_str):
    rates = [mesh_dict.get(term, [None, None, None])[2] for term in mesh_terms_str if term in mesh_dict]
    top_three_rates = sorted(filter(None, rates), reverse=True)[:3]
    last_three_rates = sorted(filter(None, rates), reverse=False)[:3]
    top_three_rates += [0] * (3 - len(top_three_rates))
    last_three_rates += [0] * (3 - len(last_three_rates))
    return top_three_rates+last_three_rates
top_rates_df = result_df['cited_mesh'].apply(get_top_three_rates).apply(pd.Series)
top_rates_df.columns = ['mesh_rate_top1', 'mesh_rate_top2', 'mesh_rate_top3','mesh_rate_last1', 'mesh_rate_last2', 'mesh_rate_last3']

top_rates_df['PMID'] = result_df['PMID']
top_rates_df.set_index('PMID', inplace=True)

In [258]:
top_rates_df

,mesh_rate_top1,mesh_rate_top2,mesh_rate_top3,mesh_rate_last1,mesh_rate_last2,mesh_rate_last3
PMID,,,,,,
24393089,0.813858,0.790030,0.767261,0.159991,0.173560,0.180219
24399715,0.366690,0.321386,0.273190,0.000298,0.000491,0.000541
24421179,0.846343,0.799674,0.758853,0.023121,0.033252,0.038730
24453941,0.334208,0.323964,0.302358,0.000168,0.000391,0.000491
24494939,0.848975,0.813858,0.747266,0.019859,0.021348,0.021639
...,...,...,...,...,...,...
30440893,0.455995,0.355821,0.345359,0.007634,0.019820,0.019892
30452750,0.989217,0.988655,0.954903,0.041012,0.050926,0.051572
30466303,0.827269,0.810378,0.802336,0.012062,0.067450,0.089694


In [259]:
top_rates_df.rename(columns={'mesh_rate_top1': 'cited_mesh_rate_top1'}, inplace=True)
top_rates_df.rename(columns={'mesh_rate_top2': 'cited_mesh_rate_top2'}, inplace=True)
top_rates_df.rename(columns={'mesh_rate_top3': 'cited_mesh_rate_top3'}, inplace=True)
top_rates_df.rename(columns={'mesh_rate_last1': 'cited_mesh_rate_last1'}, inplace=True)
top_rates_df.rename(columns={'mesh_rate_last2': 'cited_mesh_rate_last2'}, inplace=True)
top_rates_df.rename(columns={'mesh_rate_last3': 'cited_mesh_rate_last3'}, inplace=True)

In [260]:
result_df["cited_mesh_rate_top1"]=top_rates_df["cited_mesh_rate_top1"]
result_df["cited_mesh_rate_top2"]=top_rates_df["cited_mesh_rate_top2"]
result_df["cited_mesh_rate_top3"]=top_rates_df["cited_mesh_rate_top3"]
result_df["cited_mesh_rate_last1"]=top_rates_df["cited_mesh_rate_last1"]
result_df["cited_mesh_rate_last2"]=top_rates_df["cited_mesh_rate_last2"]
result_df["cited_mesh_rate_last3"]=top_rates_df["cited_mesh_rate_last3"]

In [261]:
data0.columns

Index(['PMID', 'total_sentences', 'sentence_order', 'sentences', 'candidate',
       'tagged_sentences', 'category', 'cand_length', 'cand_cap_num',
       'cand_cap_ratio', 'cand_has_num', 'cand_has_punc', 'cand_capitalized',
       'cand_words', 'cand_etal', 'comma_before', 'comma_after',
       'parentheses_before', 'parentheses_after', 'year_before', 'year_after',
       'etal_before', 'etal_after', 'cand_pos', 'og_mapaffil_res',
       'MapAffil_dict_matched', 'mapstand_state_list', 'rigion_list',
       'mapstand_country_list', 'lastname_freq', 'firstname_freq',
       'affiliation_freq', 'title_freq', 'abstract_freq',
       'lastname_freq_log10', 'firstname_freq_log10', 'affiliation_freq_log10',
       'title_freq_log10', 'abstract_freq_log10', 'cited_PMID', 'mesh',
       'unique_city', 'cited_unique_city', 'cited_unique_tempocity',
       'cited_mesh', 'mesh_rate_top1', 'mesh_rate_top2', 'mesh_rate_top3',
       'mesh_rate_last1', 'mesh_rate_last2', 'mesh_rate_last3',
       '

In [262]:
result_df['cited_mesh_rate_top1'].fillna(value=0, inplace=True)
result_df['cited_mesh_rate_top2'].fillna(value=0, inplace=True)
result_df['cited_mesh_rate_top3'].fillna(value=0, inplace=True)
result_df['cited_mesh_rate_last1'].fillna(value=0, inplace=True)
result_df['cited_mesh_rate_last2'].fillna(value=0, inplace=True)
result_df['cited_mesh_rate_last3'].fillna(value=0, inplace=True)

In [263]:
result_df.isna().any()

PMID                      False
cited_PMID                False
mesh                      False
unique_city               False
unique_tempocity          False
cited_unique_city         False
cited_unique_tempocity    False
cited_mesh                False
mesh_rate_top1            False
mesh_rate_top2            False
mesh_rate_top3            False
mesh_rate_last1           False
mesh_rate_last2           False
mesh_rate_last3           False
cited_mesh_rate_top1      False
cited_mesh_rate_top2      False
cited_mesh_rate_top3      False
cited_mesh_rate_last1     False
cited_mesh_rate_last2     False
cited_mesh_rate_last3     False
dtype: bool

In [303]:
data0=pd.read_csv("golden_self.tsv",sep="\t")

In [304]:
start_time = time.time()

data0 = pd.merge(data0, result_df, on='PMID', how='left')
end_time = time.time()
print(f"Processing time：{end_time - start_time} seconds")

Processing time：0.0039997100830078125 seconds


In [305]:
data0.isna().any()

PMID                      False
total_sentences           False
sentence_order            False
sentences                 False
candidate                 False
tagged_sentences          False
category                  False
cand_length               False
cand_cap_num              False
cand_cap_ratio            False
cand_has_num              False
cand_has_punc             False
cand_capitalized          False
cand_words                False
cand_etal                 False
comma_before              False
comma_after               False
parentheses_before        False
parentheses_after         False
year_before               False
year_after                False
etal_before               False
etal_after                False
cand_pos                  False
og_mapaffil_res           False
MapAffil_dict_matched     False
mapstand_state_list       False
rigion_list               False
mapstand_country_list     False
lastname_freq             False
firstname_freq            False
affiliat

In [306]:
data0['cited_PMID'].fillna(value='-', inplace=True)
data0['mesh'].fillna(value='-', inplace=True)
data0['unique_city'].fillna(value='[]', inplace=True)
data0['unique_tempocity'].fillna(value='[]', inplace=True)
data0['cited_unique_city'].fillna(value='[]', inplace=True)
data0['cited_unique_tempocity'].fillna(value='[]', inplace=True)
data0['cited_mesh'].fillna(value='[]', inplace=True)
data0['mesh_rate_top1'].fillna(value=0, inplace=True)
data0['mesh_rate_top2'].fillna(value=0, inplace=True)
data0['mesh_rate_top3'].fillna(value=0, inplace=True)
data0['mesh_rate_last1'].fillna(value=0, inplace=True)
data0['mesh_rate_last2'].fillna(value=0, inplace=True)
data0['mesh_rate_last3'].fillna(value=0, inplace=True)
data0['cited_mesh_rate_top1'].fillna(value=0, inplace=True)
data0['cited_mesh_rate_top2'].fillna(value=0, inplace=True)
data0['cited_mesh_rate_top3'].fillna(value=0, inplace=True)
data0['cited_mesh_rate_last1'].fillna(value=0, inplace=True)
data0['cited_mesh_rate_last2'].fillna(value=0, inplace=True)
data0['cited_mesh_rate_last3'].fillna(value=0, inplace=True)

In [307]:
data0.columns

Index(['PMID', 'total_sentences', 'sentence_order', 'sentences', 'candidate',
       'tagged_sentences', 'category', 'cand_length', 'cand_cap_num',
       'cand_cap_ratio', 'cand_has_num', 'cand_has_punc', 'cand_capitalized',
       'cand_words', 'cand_etal', 'comma_before', 'comma_after',
       'parentheses_before', 'parentheses_after', 'year_before', 'year_after',
       'etal_before', 'etal_after', 'cand_pos', 'og_mapaffil_res',
       'MapAffil_dict_matched', 'mapstand_state_list', 'rigion_list',
       'mapstand_country_list', 'lastname_freq', 'firstname_freq',
       'affiliation_freq', 'title_freq', 'abstract_freq',
       'lastname_freq_log10', 'firstname_freq_log10', 'affiliation_freq_log10',
       'title_freq_log10', 'abstract_freq_log10', 'cited_PMID', 'mesh',
       'unique_city', 'unique_tempocity', 'cited_unique_city',
       'cited_unique_tempocity', 'cited_mesh', 'mesh_rate_top1',
       'mesh_rate_top2', 'mesh_rate_top3', 'mesh_rate_last1',
       'mesh_rate_last2', 

In [308]:
data0["mapstand_country_list"][0][0]

'['

In [309]:
data0['og_mapaffil_res'] = data0['og_mapaffil_res'].apply(ast.literal_eval)
data0['mapstand_state_list'] = data0['mapstand_state_list'].apply(ast.literal_eval)
data0['rigion_list'] = data0['rigion_list'].apply(ast.literal_eval)
data0['mapstand_country_list'] = data0['mapstand_country_list'].apply(ast.literal_eval)

In [310]:
cand_in_mesh,cand_in_affil,mapcity_in_mesh,mapcity_in_affil,mapstate_in_mesh,mapstate_in_affil,mapcountry_in_mesh,mapcountry_in_affil=[],[],[],[],[],[],[],[]
stanstate_in_mesh,stancountry_in_mesh,stanregion_in_mesh=[],[],[]
# mapstand_state_list,mapstand_country_list,rigion_list
count=0
for i in range(len(data0["candidate"])):
    if i%45000==0:
        print(i,datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    flagc,flags,flagr=False,False,False
    for s in data0["mapstand_state_list"][i]:
        if len(s)>0 and (not flags):
#             print(type(s),s,merged_df["mesh"][i])
            if s in data0["mesh"][i]: flags=True
    for c in data0["mapstand_country_list"][i]:  
        if len(c)>0 and (not flagc):
            if c in data0["mesh"][i]: flagc=True
    for r in data0["rigion_list"][i]:
        if len(r)>0 and (not flagr):
            if r in data0["mesh"][i]: flagr=True
    if flags:stanstate_in_mesh.append(1)
    else:stanstate_in_mesh.append(0)
    if flagc:stancountry_in_mesh.append(1)
    else:stancountry_in_mesh.append(0)
    if flagr:stanregion_in_mesh.append(1)
    else:stanregion_in_mesh.append(0)
    
    
    flag1,flag2,flag3,flag4,flag5,flag6=False,False,False,False,False,False
#     if i%2410==0: print(i)
    if data0["candidate"][i] in data0["mesh"][i]: cand_in_mesh.append(1)
    else: cand_in_mesh.append(0)
    if (data0["candidate"][i] in data0["unique_city"][i]) or (data0["candidate"][i] in data0["unique_tempocity"][i]): cand_in_affil.append(1)
    else: cand_in_affil.append(0)
    if len(data0['og_mapaffil_res'][i])==0:
        mapcity_in_mesh.append(0)
        mapcity_in_affil.append(0)
        mapstate_in_mesh.append(0)
        mapstate_in_affil.append(0)
        mapcountry_in_mesh.append(0)
        mapcountry_in_affil.append(0)
    else:
        for res in data0['og_mapaffil_res'][i]:
            for j in range(1,4):
                if j==1:
                    if len(res[j])>0 and not flag1:
                        if res[j] in data0["mesh"][i]:flag1=True   
                    if len(res[j])>0 and not flag2:
                        if (res[j] in data0["unique_city"][i]) or (res[j] in data0["unique_tempocity"][i]):flag2=True        

                if j==2:
                    if len(res[j])>0 and not flag3:
                        if res[j] in data0["mesh"][i]:flag3=True   
                    if len(res[j])>0 and not flag4:
                        if (res[j] in data0["unique_city"][i]) or (res[j] in data0["unique_tempocity"][i]):flag4=True   

                if j==3:
                    if len(res[j])>0 and not flag5:
                        if res[j] in data0["mesh"][i]:flag5=True  
                    if len(res[j])>0 and not flag6:        
                        if (res[j] in data0["unique_city"][i]) or (res[j] in data0["unique_tempocity"][i]):flag6=True  

        if flag1:mapcity_in_mesh.append(1)
        else:mapcity_in_mesh.append(0)
        if flag2:mapcity_in_affil.append(1)
        else:mapcity_in_affil.append(0)
        if flag3:mapstate_in_mesh.append(1)
        else:mapstate_in_mesh.append(0)
        if flag4:mapstate_in_affil.append(1)
        else:mapstate_in_affil.append(0)
        if flag5:mapcountry_in_mesh.append(1)
        else:mapcountry_in_mesh.append(0)
        if flag6:mapcountry_in_affil.append(1)
        else:mapcountry_in_affil.append(0)
            

0 2024-05-02 13:05:03


In [311]:
data0["cand_in_mesh"]=cand_in_mesh
data0["cand_in_affil"]=cand_in_affil
data0["mapcity_in_mesh"]=mapcity_in_mesh
data0["mapcity_in_affil"]=mapcity_in_affil
data0["mapstate_in_mesh"]=mapstate_in_mesh
data0["mapstate_in_affil"]=mapstate_in_affil
data0["mapcountry_in_mesh"]=mapcountry_in_mesh
data0["mapcountry_in_affil"]=mapcountry_in_affil

In [312]:
data0["stanstate_in_mesh"]=stanstate_in_mesh
data0["stancountry_in_mesh"]=stancountry_in_mesh
data0["stanregion_in_mesh"]=stanregion_in_mesh

In [313]:
data0.columns

Index(['PMID', 'total_sentences', 'sentence_order', 'sentences', 'candidate',
       'tagged_sentences', 'category', 'cand_length', 'cand_cap_num',
       'cand_cap_ratio', 'cand_has_num', 'cand_has_punc', 'cand_capitalized',
       'cand_words', 'cand_etal', 'comma_before', 'comma_after',
       'parentheses_before', 'parentheses_after', 'year_before', 'year_after',
       'etal_before', 'etal_after', 'cand_pos', 'og_mapaffil_res',
       'MapAffil_dict_matched', 'mapstand_state_list', 'rigion_list',
       'mapstand_country_list', 'lastname_freq', 'firstname_freq',
       'affiliation_freq', 'title_freq', 'abstract_freq',
       'lastname_freq_log10', 'firstname_freq_log10', 'affiliation_freq_log10',
       'title_freq_log10', 'abstract_freq_log10', 'cited_PMID', 'mesh',
       'unique_city', 'unique_tempocity', 'cited_unique_city',
       'cited_unique_tempocity', 'cited_mesh', 'mesh_rate_top1',
       'mesh_rate_top2', 'mesh_rate_top3', 'mesh_rate_last1',
       'mesh_rate_last2', 

In [314]:
cand_in_mesh,cand_in_affil,mapcity_in_mesh,mapcity_in_affil,mapstate_in_mesh,mapstate_in_affil,mapcountry_in_mesh,mapcountry_in_affil=[],[],[],[],[],[],[],[]
stanstate_in_mesh,stancountry_in_mesh,stanregion_in_mesh=[],[],[]
# mapstand_state_list,mapstand_country_list,rigion_list
count=0
for i in range(len(data0["candidate"])):
    if i%46672==0:
        print(i,datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    flagc,flags,flagr=False,False,False
    for s in data0["mapstand_state_list"][i]:
        if len(s)>0 and (not flags):
#             print(type(s),s,merged_df["mesh"][i])
            if s in data0["cited_mesh"][i]: flags=True
    for c in data0["mapstand_country_list"][i]:  
        if len(c)>0 and (not flagc):
            if c in data0["cited_mesh"][i]: flagc=True
    for r in data0["rigion_list"][i]:
        if len(r)>0 and (not flagr):
            if r in data0["cited_mesh"][i]: flagr=True
    if flags:stanstate_in_mesh.append(1)
    else:stanstate_in_mesh.append(0)
    if flagc:stancountry_in_mesh.append(1)
    else:stancountry_in_mesh.append(0)
    if flagr:stanregion_in_mesh.append(1)
    else:stanregion_in_mesh.append(0)
    
    
    flag1,flag2,flag3,flag4,flag5,flag6=False,False,False,False,False,False
#     if i%2410==0: print(i)
    if data0["candidate"][i] in data0["cited_mesh"][i]: cand_in_mesh.append(1)
    else: cand_in_mesh.append(0)
    if data0["candidate"][i] in data0["cited_unique_tempocity"][i]: cand_in_affil.append(1)
    else: cand_in_affil.append(0)
    if len(data0['og_mapaffil_res'][i])==0:
        mapcity_in_mesh.append(0)
        mapcity_in_affil.append(0)
        mapstate_in_mesh.append(0)
        mapstate_in_affil.append(0)
        mapcountry_in_mesh.append(0)
        mapcountry_in_affil.append(0)
    else:
        for res in data0['og_mapaffil_res'][i]:
            for j in range(1,4):
                if j==1:
                    if len(res[j])>0 and not flag1:
                        if res[j] in data0["cited_mesh"][i]:flag1=True   
                    if len(res[j])>0 and not flag2:
                        if res[j] in data0["cited_unique_tempocity"][i]:flag2=True        

                if j==2:
                    if len(res[j])>0 and not flag3:
                        if res[j] in data0["cited_mesh"][i]:flag3=True   
                    if len(res[j])>0 and not flag4:
                        if res[j] in data0["cited_unique_tempocity"][i]:flag4=True   

                if j==3:
                    if len(res[j])>0 and not flag5:
                        if res[j] in data0["cited_mesh"][i]:flag5=True  
                    if len(res[j])>0 and not flag6:        
                        if res[j] in data0["cited_unique_tempocity"][i]:flag6=True  

        if flag1:mapcity_in_mesh.append(1)
        else:mapcity_in_mesh.append(0)
        if flag2:mapcity_in_affil.append(1)
        else:mapcity_in_affil.append(0)
        if flag3:mapstate_in_mesh.append(1)
        else:mapstate_in_mesh.append(0)
        if flag4:mapstate_in_affil.append(1)
        else:mapstate_in_affil.append(0)
        if flag5:mapcountry_in_mesh.append(1)
        else:mapcountry_in_mesh.append(0)
        if flag6:mapcountry_in_affil.append(1)
        else:mapcountry_in_affil.append(0)

0 2024-05-02 13:05:13


In [315]:
data0["cand_in_cited_mesh"]=cand_in_mesh
data0["cand_in_cited_affil"]=cand_in_affil
data0["mapcity_in_cited_mesh"]=mapcity_in_mesh
data0["mapcity_in_cited_affil"]=mapcity_in_affil
data0["mapstate_in_cited_mesh"]=mapstate_in_mesh
data0["mapstate_in_cited_affil"]=mapstate_in_affil
data0["mapcountry_in_cited_mesh"]=mapcountry_in_mesh
data0["mapcountry_in_cited_affil"]=mapcountry_in_affil
data0["stanstate_in_cited_mesh"]=stanstate_in_mesh
data0["stancountry_in_cited_mesh"]=stancountry_in_mesh
data0["stanregion_in_cited_mesh"]=stanregion_in_mesh

In [316]:
data0.columns

Index(['PMID', 'total_sentences', 'sentence_order', 'sentences', 'candidate',
       'tagged_sentences', 'category', 'cand_length', 'cand_cap_num',
       'cand_cap_ratio', 'cand_has_num', 'cand_has_punc', 'cand_capitalized',
       'cand_words', 'cand_etal', 'comma_before', 'comma_after',
       'parentheses_before', 'parentheses_after', 'year_before', 'year_after',
       'etal_before', 'etal_after', 'cand_pos', 'og_mapaffil_res',
       'MapAffil_dict_matched', 'mapstand_state_list', 'rigion_list',
       'mapstand_country_list', 'lastname_freq', 'firstname_freq',
       'affiliation_freq', 'title_freq', 'abstract_freq',
       'lastname_freq_log10', 'firstname_freq_log10', 'affiliation_freq_log10',
       'title_freq_log10', 'abstract_freq_log10', 'cited_PMID', 'mesh',
       'unique_city', 'unique_tempocity', 'cited_unique_city',
       'cited_unique_tempocity', 'cited_mesh', 'mesh_rate_top1',
       'mesh_rate_top2', 'mesh_rate_top3', 'mesh_rate_last1',
       'mesh_rate_last2', 

In [317]:
data0.to_csv("golden_engi_self_cited_all.tsv",sep='\t',index=False)

In [318]:
data0=data0.drop(['sentences','cited_PMID','og_mapaffil_res','unique_city','unique_tempocity','mapstand_state_list','mapstand_country_list','rigion_list','mesh','candidate','cited_unique_tempocity', 'cited_mesh'], axis=1)

In [319]:
data0.to_csv('golden_engi_self_cited_all_colab.tsv', sep='\t',index=False)

In [320]:
data0.to_excel('golden_engi_self_cited_all_colab.xlsx', index=False)